In [64]:
import numpy as np
import pandas as pd
from pandas import read_csv

import os, sys
from os.path import dirname
from cmdstanpy import CmdStanModel,install_cmdstan


ROOT_DIR = '.'


In [13]:
#import data
raw_data = read_csv(os.path.join(ROOT_DIR,'data','data.csv'))
data = raw_data.copy()
# data rejections
reject = read_csv(os.path.join(ROOT_DIR,'data','reject.csv'))
data = data[data.subject.isin(reject.query('reject==0').subject)]

## Sort data.
f = lambda x: x.subject + '_' + x.rune
data['stimulus'] = np.unique(data.apply(f, 1), return_inverse=True)[-1] + 1
data = data.sort_values(['stimulus','exposure']).reset_index(drop=True)

## Reformat columns.
data['w1'] = np.logical_and(data.choice == 1, data.outcome == 10).astype(int)
data['w2'] = np.logical_and(data.choice == 0, data.outcome == -10).astype(int)
data['w3'] = 1 - np.logical_or(data.w1, data.w2).astype(int)
data['outcome'] = data.outcome.replace({10:1, 1:0, -1:1, -10:0})
data['valence'] = data.valence.replace({'win':1, 'lose':0})

In [14]:
raw_data[['action','choice']]

,action,choice
0,go,1
1,no-go,0
2,no-go,1
3,go,1
4,go,1
...,...,...
84475,go,1
84476,go,1
84477,no-go,0
84478,go,1


In [7]:

#raw_data.iloc[0:50][['robot', 'action', 'correct', 'choice', 'accuracy']]
#set(( raw_data.choice == raw_data.correct ) == raw_data.accuracy)

#set(raw_data[(raw_data.action=='no-go')]['robot'])#, 'action', 'correct', 'choice', 'accuracy']]

#raw_data.iloc[0:50][['exposure', 'stimulus']]


data

subject         object         subject id 
session         int64          session id {1, 2, 3}
block           int64          block id {1, 2}
runsheet       object          ? {'1a', '1b', '2a', '2b', '3a', '3b'}
trial           int64          trial id \[1, 240\]
exposure        int64          stim exposure (increasing) \[1- 12\]
valence        object          trial context, if lose- robots are GAL/NGAL. if win- robots are GW/NGW {'lose', 'win'}
action         object          ? {'go', 'no-go'}
robot          object          robot type {'GAL', 'GW', 'NGAL', 'NGW'}
stimulus      float64          robot stim \[0-11\]
rune           object          ? \[A-Z\]
rune_set       object          ? {'bacs1', 'bacs2', 'elianto'}
correct         int64          the actual correct response where 1 is go, 0 is NG{0,1}
choice          int64          subject's response  {0,1}
rt            float64          RT (s?) 
accuracy        int64          the accuracy of the response (incorrect/correct) {0,1}
sham          float64          ? {0,1}
outcome         int64          reward {-10, -1, 1, 10}
total_keys      int64          ? {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, 18}
    



In [15]:
## Define metadata.
N = data.stimulus.nunique()
M = data.exposure.nunique()
J = np.unique(data[data.exposure==1].subject, return_inverse=True)[-1] + 1

## Prepare censored mappings.
C = data.pivot_table('choice','exposure','stimulus').notnull().values.astype(int)
C = np.where(data.pivot_table('rt','exposure','stimulus',dropna=False).values < 0.2, 0, C)

## Prepare task variables.
R = data.pivot_table('outcome','exposure','stimulus').fillna(0).values
V = data.pivot_table('valence','exposure','stimulus').fillna(0).values
W = data.pivot_table(['w1','w2','w3'],'stimulus','exposure').fillna(0).values.reshape(N,-1,M).T

## Prepare response variables.
Y = data.pivot_table('choice','exposure','stimulus').fillna(0).values.astype(int)


In [65]:
install_cmdstan()

Installing CmdStan version: 2.30.0
Install directory: /Users/gilikarni/.cmdstan
Download successful, file: /var/folders/zh/4y0xxj8j5ps5r40hdx73zwtw0000gn/T/tmp_twgzaq3
Unpacked download as cmdstan-2.30.0
Building version cmdstan-2.30.0
Test model compilation
Installed cmdstan-2.30.0
deleting tmpfiles dir: /var/folders/zh/4y0xxj8j5ps5r40hdx73zwtw0000gn/T/tmpoyie6oer
done



True

In [66]:
StanModel = CmdStanModel(stan_file=os.path.join(ROOT_DIR,'stan_models','Hanneke_m1.stan'))


INFO:cmdstanpy:compiling stan program, exe file: /Users/gilikarni/Documents/RobotFactory/study02/stan_models/Hanneke_m1
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
ERROR:cmdstanpy:file /Users/gilikarni/Documents/RobotFactory/study02/stan_models/Hanneke_m1.stan, exception ERROR
error: PCH file was compiled for the target 'x86_64-apple-macosx11.0.0' but the current translation unit is being compiled for target 'x86_64-apple-macosx11.3.0'
1 error generated.
make: *** [/Users/gilikarni/Documents/RobotFactory/study02/stan_models/Hanneke_m1] Error 1 
ERROR:cmdstanpy:model compilation failed


ValueError: Unable to compile Stan model file: /Users/gilikarni/Documents/RobotFactory/study02/stan_models/Hanneke_m1.stan.

In [31]:
C = data.pivot_table('choice','exposure','stimulus').notnull().values.astype(int)
C = np.where(data.pivot_table('rt','exposure','stimulus',dropna=False).values < 0.2, 0, C)


In [35]:
 data.pivot_table('choice','exposure','stimulus').notnull()
    
    
    

stimulus,1,2,3,4,5,6,7,8,9,10,...,2839,2840,2841,2842,2843,2844,2845,2846,2847,2848
exposure,,,,,,,,,,,,,,,,,,,,,
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,False,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
